# Interpolación a trozos

## Introducción

A medida que el número de puntos a interpolar aumenta, el spline o polinomio interpolante a trozos suele convertirse en la mejor opción. Es considerablemente más robusto que un polinomio en el sentido de que tiene menos tendencia a oscilar entre nodos.

Considera el problema de aproximar una función $f(x)$ en un intervalo $[a,b]$. Mediante los $n+1$ puntos (nodos) $a = x_0 < x_1 < \cdots < x_n = b$ el intervalo $[a,b]$ se divide en $n$ subintervalos, no necesariamente del mismo tamaño. En dichos nodos, el valor de la función es conocido, $f(x_i) = y_i$. En cada subintervalo, el polinomio interpolante a trozos $q(x)$ tomará una forma distinta:
$$
q(x) = 
\left\{
    \begin{array}{cc}
        q_{0,1}(x) & \text{si } x_0 < x \le x_1 \\
        q_{1,2}(x) & \text{si } x_1 < x \le x_2 \\
        \vdots & \vdots \\
        q_{n-1,n}(x) & \text{si } x_{n-1} < x \le x_n
    \end{array}
\right.
$$


<center>
<div>
<img src="https://drive.upm.es/index.php/apps/files_sharing/ajax/publicpreview.php?x=1920&y=465&a=true&file=splines.jpg&t=JZNROFMw49WDrlu&scalingup=0" width="450"/>
</div>
</center>


Observa que usamos la notación $q_{i,i+1}(x)$ para referirnos al polinomio que se extiende desde el nodo $i$ al $i+1$. En general, cada $q_{i,i+1}$ tendrá una expresión polinómica distinta. Se llama *spline de grado $r$* si el mayor grado de estos polinomios $q_{i,i+1}$ es $r$. El valor más usado es $r=3$, de manera que $q(x)$ sería un polinomio cúbico en cada subintervalo. Pero hay otras opciones. 

## Interpolación lineal a trozos

El caso más simple de interpolación polinómica a trozos es la "línea poligonal", es decir, cada par de puntos $(x_i, y_i)$ y $(x_{i+1}, y_{i+1})$ se unen por una linea recta. De esta manera, el polinomio interpolante a trozos $q(x)$ es un polinomio de primer grado en cada subintervalo $[x_i, x_{i+1}]$, y verifica que $q(x_i) = y_i$ y que es continuo ($q_{i-1,i}(x_i) = q_{i,i+1}(x_i)$). Se deduce de forma trivial que la expresión matemática de un $q_{i, i+1}$ genérico será:
$$
q_{i, i+1}(x)=y_i+\frac{y_{i+1}-y_{i}}{x_{i+1}-x_i}\left(x-x_i\right)
$$

En la siguiente figura se muestra un ejemplo en el que se interpolan los datos de un experimento por un polinomio de grado 10 y por un polinomio a trozos de primer grado. Claramente, salvo que la función sea muy peculiar, la línea poligonal es una aproximación más aceptable que el polinomio interpolante.

<center>
<div>
<img src="https://drive.upm.es/index.php/apps/files_sharing/ajax/publicpreview.php?x=1920&y=465&a=true&file=linea_poligonal.jpg&t=I3unQpcrnoTedY2&scalingup=0" width="450"/>
</div>
</center>


La línea poligonal presenta el problema de no ser suave (su derivada es discontinua) lo cual tampoco es un serio inconveniente en muchos casos. Las funciones spline que se verán a continuación resuelven este problema.

Además de su simplicidad, otra ventaja de la aproximación polinómica a trozos es la siguiente: si la función $f(x)$ tiene un mal comportamiento en el entorno de algún punto, la "mejor" aproximación polinómica (Lagrange) es muy probable que sea una mala aproximación en todo el intervalo $[a, b]$. Sin embargo, con una aproximación polinómica a trozos, eligiendo los puntos $x_i$ apropiadamente es posible confinar el mal comportamiento en un intervalo próximo al punto problemático y tener una buena aproximación en el resto del intervalo $[a, b]$.

## Splines cúbicos

Si en cada subintervalo se utilizan polinomios de grado 3, el polinomio interpolante se llama *spline cúbico*. Tendrá la forma:

$$
q_{i, i+1} = a_i + b_i x + c_i x^2 + d_i x^3
$$

De manera que cada subintervalo introduce $4$ incógnitas a determinar: $\left\{ a_i, b_i, c_i, d_i \right\}$. Como hay en total $n$ subintervalos, se deberán calcular $4n$ incógnitas, para las que harán falta $4n$ ecuaciones. 

 - De la condición de interpolación $y_i = q_{i,i+1} (x_i)$ en cada uno de los $n+1$ nodos se obtienen $n +1$ ecuaciones.

 - De la condición de continuidad de la función en los nodos internos $q_{i-1,i} (x_i) = q_{i,i+1} (x_i)$ para $i = 1, \dots, n-1$ se obtienen otras $n-1$ condiciones.

 - Los splines cúbicos imponen también condición de continuidad en la primera y segunda derivada en los nodos. De esta forma se tendrá también que $q'_{i-1,i} (x_i) = q'_{i,i+1} (x_i)$ y $q''_{i-1,i} (x_i) = q''_{i,i+1} (x_i)$ para $i = 1, \dots, n-1$, que introducen $(n-1) + (n-1)$ ecuaciones adicionales.

Con todo, se disponen de $(n+1) + (n-1) + (n-1) + (n-1)= 4n -2$ ecuaciones. Por lo que es necesario imponer dos ecuaciones adicionales para poder resolver el problema. Estas ecuaciones se llaman a veces *condiciones de cierre*. Normalmente, dichas ecuaciones hacen referencia al comportamiento de la función en los nodos extremos $x_0$ y $x_n$.

Existen varias opciones. Una elección posible es tomar $q''(x_0) = q''(x_n) = 0$. El spline resultante recibe el nombre de _spline cúbico natural_. Otra elección muy utilizada es hacer $q'(x_0) = f'(x_0)$ y $q'(x_n) = f'(x_n)$, si se conoce (o aproxima) la derivada de la función original en los extremos, dando lugar al _spline cúbico de Hermite_. Hay también otras opciones.

### Cálculo efectivo del spline cúbico

A continuación se propone una forma eficiente de organizar el cálculo de las $4n$ incógnitas de un spline cúbico. Se tomará como condiciones de cierre las del **spline cúbico natural** ($q''(x_0) = q''(x_n) = 0$), pero el procedimiento descrito a continuación se puede adaptar fácilmente a otras condiciones. 

La condición de continuidad para la segunda derivada establece que:
$$
q_{i-1, i}^{\prime \prime}\left(x_i\right)=q_{i, i+1}^{\prime \prime}\left(x_i\right)=k_i
$$
En este punto, todos los $k_i$ son desconocidos, excepto:
$$
k_0 = k_n = 0
$$
por las condiciones de cierre consideradas.


El punto de partida para el cálculo de los coeficientes de $q_{i, i+1}$ es la expresión de su derivada segunda, $q''_{i, i+1}$, que sabemos que debe ser lineal. Usando la fórmula de interpolación de Lagrange con dos puntos, podemos escribir:
$$
q''_{i, i+1} = k_i \cdot l_i (x) + k_{i+1} \cdot l_{i+1} (x)
$$
donde 
$$
l_i(x)=\frac{x-x_{i+1}}{x_i-x_{i+1}} \quad l_{i+1}(x)=\frac{x-x_i}{x_{i+1}-x_i}
$$
Sustituyendo:
$$
q_{i, i+1}^{\prime \prime}(x)=\frac{k_i\left(x-x_{i+1}\right) - k_{i+1}\left(x-x_i\right)}{x_i -x_{i+1}}
$$
Integrando dos veces con respecto a $x$, obtenemos:
$$
q_{i, i+1}(x)=\frac{k_i\left(x-x_{i+1}\right)^3 - k_{i+1}\left(x-x_i\right)^3}{6(x_i - x_{i+1})}+A\left(x-x_{i+1}\right)-B\left(x-x_i\right)
$$
donde $A$ y $B$ son constantes de integración. Los términos resultantes de la integración normalmente se escribirían como $Cx + D$. Haciendo $C = A - B$ y $D = -A x_i + 1 + B x_i$, se obtienen los dos últimos términos de la ecuación anterior, que simplificarán las expresiones en los cálculos que siguen.


Imponiendo ahora la condición de interpolación, $q_{i,i+1} (x_i)= y_i$ obtenemos:
$$
\frac{k_i\left(x_i-x_{i+1}\right)^3}{6\left(x_i-x_{i+1}\right)}+A\left(x_i-x_{i+1}\right)=y_i
$$
Y por tanto,
$$
A=\frac{y_i}{x_i - x_{i+1}}-\frac{k_i}{6} \cdot (x_i - x_{i+1})
$$
De forma análoga con $q_{i,i+1} (x_{i+1}) = y_{i+1}$ (condiciones de continuidad):
$$
B=\frac{y_{i+1}}{x_i - x_{i+1}}-\frac{k_{i+1}}{6} \cdot (x_i - x_{i+1})
$$
Sustituyendo en la expresión anterior:
$$
\begin{aligned}
q_{i, i+1}(x) = & \frac{k_i}{6}\left[\frac{\left(x-x_{i+1}\right)^3}{x_i - x_{i+1}}- \left(x-x_{i+1}\right) \cdot \left(x_i - x_{i+1}\right) \right] \\
& -\frac{k_{i+1}}{6}\left[\frac{\left(x-x_i\right)^3}{x_i - x_{i+1}}- \left(x-x_i\right) \cdot \left(x_i - x_{i+1} \right) \right] \\
& +\frac{y_i\left(x-x_{i+1}\right)-y_{i+1}\left(x-x_i\right)}{x_i - x_{i+1}}
\end{aligned}
$$

Imponiendo ahora las condiciones de continuidad de la derivada ($q'_{i-1,i} (x_i) = q'_{i,i+1} (x_i)$) y operando se llega al siguiente sistema de ecuaciones:
$$
\left(x_{i-1} - x_{i}\right) k_{i-1} + 2 \left(x_{i-1} - x_{i+1} \right) k_i + \left(x_i - x_{i+1}\right) k_{i+1} = 6 \left(\frac{y_{i-1}-y_{i}}{x_{i-1} - x_i} - \frac{y_{i}-y_{i+1}}{x_{i} - x_{i+1}}\right), \quad i=1,2, \cdots, n-1
$$
Si la malla es de paso constante $x_{i-1} - x_{i} = x_{i} - x_{i+1} = -h$, la expresión anterior puede simplificarse a:
$$
k_{i-1} + 4 k_i + k_{i+1} = \frac{6}{h^2} \left( y_{i-1} - 2y_i + y_{i+1} \right), \quad i=1,2, \cdots, n-1
$$

### Forma matricial

El sistema anterior puede expresarse matricialmente como
$$
A k=b
$$
$$
A=\left[\begin{array}{cccccc}
1 & 0 & 0 & 0 & \cdots & 0 \\
\left(x_{0} - x_{1}\right) &  2 \left(x_{0} - x_{2} \right) & \left(x_1 - x_{2}\right) & & & \vdots \\
0 & \left(x_{1} - x_{2}\right) &  2 \left(x_{1} - x_{3} \right) & \left(x_2 - x_{3}\right) & & \vdots \\
0 & & & \ddots & & \vdots \\
\vdots & & & \left(x_{n-2} - x_{n-1}\right) &  2 \left(x_{n-2} - x_{n} \right) & \left(x_{n-1} - x_{n}\right) \\
0 & & & & 0 & 1
\end{array}\right]
$$
$$
b=\left(0, 6 \left( \frac{y_0-y_1}{x_0-x_1}-\frac{y_1-y_2}{x_1-x_2} \right), \ldots, 6 \left( \frac{y_{i-1}-y_i}{x_{i-1}-x_i}-\frac{y_{i}-y_{i+1}}{x_{i}-x_{i+1}} \right), \ldots, 6 \left( \frac{y_{n-2}-y_{n-1}}{x_{n-2}-x_{n-1}}-\frac{y_{n-1}-y_n}{x_{n-1}-x_n} \right), 0\right)^{\top}
$$
$$
k=\left(k_0, k_1, \ldots, k_n\right)^{\top}
$$

Recuerda que se está asumiendo que las condiciones de cierre son que la segunda derivada es nula en los extremos del dominio (i.e. $k_0 = k_n = 0$).

Una vez obtenidos los $k_i$, se puede **recuperar el polinomio** sustituyendo en la ecuación de la celda anterior para $q_{i, i+1}(x)$

**Ejercicio 1 -** Calcula el spline cúbico que interpola los siguientes datos, en dos situaciones:
 - **a.** Sabiendo que $q''(0) = q''(3) = 0$ (spline natural)
 - **b.** Sabiendo que $q'(0) = q'(3) = 1$ (spline cúbico de Hermite)

<center>

| $x$ | $y$ |
|-----|-----|
| 0 | 1 |
| 1 | 3 |
| 2 | 2 |
| 3 | 5 |

</center>


inicializa A con una  matriz de ceros porque la matriz de la estructura tiene muchos ceros

In [9]:
import numpy as np

def find_interval(x_data: list, x_val: float) -> int | None:

    for i in range(len(x_data)-1):
        if x_data[i] <= x_val <= x_data[i + 1]:
            return i
        
    return None


def evaluate_cubic_spline(x_data: list,
                          y_data: list,
                          x_val: float) -> float:

    total_nodes = len(x_data)

    assert len(x_data) == len(y_data), "las dimensiones de los datos no son correctos"    

    #A es una matriz de coeficientes
    A = np.zeros((len(x_data), len(x_data)))

    #al primera columna tiene un 1 y el resto 0s
    #al ultima columna tiene todo 0s y el ultimo 1
    A[0,0] = 1
    A[-1, -1] = 1

    # tiene qie ir de 1 a n-1 proque no queremos modificar ni la primera ni la ultima linea
    for i in range(1, len(x_data) - 1):

        A[i, i-1] = x_data[i-1] - x_data[i]
        A[i, i] = 2 ** (x_data[i-1] - x_data[i+1])
        A[i, i+1] = x_data[i] - x_data[i + 1]


    b = np.zeros(len(x_data))
    for i in range(1, len(x_data) - 1):
        b[i] = 6.0 * (((y_data[i-1] - y_data[i]) / (x_data[i-1] - x_data[i])) - ((y_data[i] - y_data[i+1]) / (x_data[i] - x_data[i + 1])))


    k = np.linalg.solve(A, b)

    #term1 = (k[i]/6) * ( (x_val - x_data[i+1]) ** 3 / (x_data[i] - x_data[i + 1]) - (x_val - x_data[i + 1]) * (x_data[i] -  x_data[i + 1]))
    #term2 = - (k[i+1] / 6.0) * ( (x_val - x_data[i])**3 / (x_data[i] - x_data[i+1]) - (x_val - x_data[i+1] * (x_data[i] - x_data[i+1])))
    #term3 = (y_data[i] * (x_val - x_data[i+1]) - y_data[i+1] * (x_val - x_data[i])) / (x_data[i]- x_data[i+1])

    term1 = (k[i] / 6.0) * ( (x_val - x_data[i+1])**3 / (x_data[i] - x_data[i+1]) - (x_val - x_data[i+1]) * (x_data[i] - x_data[i+1]))
    term2 = - ((k[i+1] / 6.0) * ( (x_val - x_data[i])**3 / (x_data[i] - x_data[i+1]) - (x_val - x_data[i]) * (x_data[i] - x_data[i+1])))
    term3 = (y_data[i] * (x_val - x_data[i+1]) - y_data[i+1] * (x_val - x_data[i])) / (x_data[i] - [i+1])

    q = term1 + term2 + term3
    return q
    print(A)

In [2]:
%pip install spicy
from scipy.interpolate import CubicSpline

^C
Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/44.5 MB ? eta -:--:--
    --------------------------------------- 0.8/44.5 MB 2.6 MB/s eta 0:00:17
   - -------------------------------------- 1.3/44.5 MB 2.6 MB/s eta 0:00:17
   - -------------------------------------- 1.8/44.5 MB 2.7 MB/s eta 0:00:16
   -- ------------------------------------- 2.6/44.5 MB 2.8 MB/s eta 0:00:15
   -- ------------------------------------- 3.1/44.5 MB 2.9 MB/s eta 0:00:15
   --- ------------------------------------ 3.9/44.5 MB 2.9 MB/s eta 0:00:15
   ---- ----------------------------------- 4.5/44.5 MB 2.9 MB/s eta 0:00:14
   ---- ----------------------------------- 5.2/44.5 MB 3.0 MB/s eta 0:00:14
   ----- ---------------------------------- 5.8/44.5 MB 3.0 MB/s eta 0:00:14
   ----- ---------------------------------- 6.6/44.5 MB 3.0 MB/s eta 0:00:13
   ------ --------------------------------- 7.1/44.5 MB 3.0 MB/s eta 0:00:13
   ------ ---

In [10]:
x_data = [0, 1, 2, 3]
y_data = [1, 3, 2, 5]

evaluate_cubic_spline(x_data, y_data, 0.5)

TypeError: unsupported operand type(s) for -: 'int' and 'list'